In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

# Define the neural network model
class ThreeLayerClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(ThreeLayerClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Define the dataset
df = pd.read_csv("../data-gathering/extracted/erwin.csv")
target_tensor = torch.tensor(df["class"].values, dtype=torch.float32).unsqueeze(1)
input_tensor = df.drop(df.columns[0], axis=1)
input_tensor = torch.tensor(input_tensor.values, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(input_tensor, target_tensor)

# Initialize the DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
input_size = 132
hidden_size1 = 50
hidden_size2 = 25
output_size = 1

model = ThreeLayerClassifier(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 600

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
    
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print progress
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save only the model state dictionary (recommended)
torch.save(model.state_dict(), 'erwin.pth')

# Test the model with a new input
new_input = torch.rand(1, 132)
predicted_output = model(new_input)
predicted_class = 1 if predicted_output.item() > 0.5 else 0

print(f'New input: {new_input.numpy()}')
print(f'Predicted output: {predicted_output.item()}, Predicted class: {predicted_class}')


/home/muzakki/anaconda3/envs/smart_prison/lib/python3.9/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [100/600], Loss: 0.0008
Epoch [200/600], Loss: 0.0054
Epoch [300/600], Loss: 0.0061
Epoch [400/600], Loss: 0.0010
Epoch [500/600], Loss: 0.0000
Epoch [600/600], Loss: 0.0000
New input: [[0.17778254 0.2559353  0.41094768 0.91213256 0.57191485 0.06201434
  0.75466967 0.58961445 0.63055897 0.05914634 0.3310938  0.82341087
  0.37916017 0.649397   0.403677   0.21829289 0.5715275  0.8848691
  0.15404916 0.85253227 0.91840553 0.8035078  0.508107   0.37005806
  0.9210604  0.5415436  0.7915857  0.92165744 0.22165829 0.7729926
  0.06512827 0.9941835  0.8202588  0.15674478 0.1954189  0.02929294
  0.34319568 0.7240504  0.7887502  0.240215   0.62214184 0.09707969
  0.88881844 0.08177131 0.02675891 0.05565137 0.53726333 0.18109435
  0.9630627  0.5424279  0.44097626 0.99478453 0.2705713  0.08638573
  0.61121476 0.8842615  0.3479954  0.75360274 0.69259685 0.97032624
  0.6972656  0.01168072 0.47648472 0.05483973 0.67319244 0.48182142
  0.39890307 0.33174938 0.32010245 0.08711243 0.61131877 0.0403

# SCIKIT-LEARN

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib

# Load and preprocess the dataset
df = pd.read_csv("../data-gathering/extracted/with_visibility/person1.csv")
target = df["class"].values
features = df.drop(df.columns[0], axis=1).values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train KNN
knn_model = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn_model.fit(X_train_scaled, y_train)

# Save the model
model_filename = 'ML/person1.joblib'
joblib.dump(knn_model, model_filename)
print(f'Model saved as {model_filename}')

# Make predictions on test data
y_pred = knn_model.predict(X_test_scaled)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Load the model
loaded_model = joblib.load(model_filename)

# You can now use loaded_model for predictions


In [ ]:
import pandas as pd
import numpy as np

# Generate random data for the DataFrame
np.random.seed(42)
num_samples = 100

data = {
    'class': torch.randint(0, 2, (num_samples,)),
}

for i in range(1 ,34):
    data[f"x{i}"] = torch.rand(num_samples)
    data[f"y{i}"] = torch.rand(num_samples)
    data[f"z{i}"] = torch.rand(num_samples)

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df.head())


In [ ]:
import pandas as pd
import torch

# Your input data
input_data = [
    {'x': 0.4978089928627014, 'y': 0.43863680958747864, 'z': -0.22711889445781708, 'visibility': 0.9999940395355225},
    {'x': 0.5025001764297485, 'y': 0.430571585893631, 'z': -0.21745352447032928, 'visibility': 0.9999876022338867},
    {'x': 0.5049259066581726, 'y': 0.4312465190887451, 'z': -0.2174386978149414, 'visibility': 0.9999890327453613},
    # Add more data as needed
]

# Convert input data to DataFrame
df_data = {'class': torch.randint(0, 2, (len(input_data),))}
for i, entry in enumerate(input_data, start=1):
    df_data[f'x{i}'] = entry['x']
    df_data[f'y{i}'] = entry['y']
    df_data[f'z{i}'] = entry['z']

# Create the DataFrame
df = pd.DataFrame(df_data)

# Display the DataFrame
print(df.head())

In [ ]:
import pandas as pd
import torch

# Create a sample DataFrame
data = {
    'class': torch.randint(0, 2, (5,)),
    'x1': torch.rand(5),
    'y1': torch.rand(5),
    'z1': torch.rand(5),
    'x2': torch.rand(5),
    'y2': torch.rand(5),
    'z2': torch.rand(5),
}

df = pd.DataFrame(data)

# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Check the type of df
print(f"Type of df: {type(df)}")

# Add a new row to the DataFrame
new_row = {
    'class': torch.randint(0, 2, (1,)).item(),
    'x1': torch.rand(1).item(),
    'y1': torch.rand(1).item(),
    'z1': torch.rand(1).item(),
    'x2': torch.rand(1).item(),
    'y2': torch.rand(1).item(),
    'z2': torch.rand(1).item(),
}

# Check the type of new_row
print(f"Type of new_row: {type(new_row)}")

# Use DataFrame's append method
df = df.append(new_row, ignore_index=True)

# Display the DataFrame with the new row
print("\nDataFrame with New Row:")
print(df)
